In [128]:
import os    
from copy import deepcopy
import codecs
import glob
import csv
import re
import numpy as np
import random
from sklearn.base import TransformerMixin
from sklearn.pipeline import make_pipeline, make_union
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.cross_validation import train_test_split


In [2]:
files = os.listdir('./data/')

In [3]:
deep_list = ['/*' * count for count in range(10)]

In [82]:
ext_list = [".gcc", ".c",".sbcl",".hack", ".php",".jruby",".yarv",".racket",".clojure",
            ".csharp",".java",".ocaml",".perl",".javascript",".python3",".scala",".ghc"]

In [83]:
this = [(glob.glob('./data/benchmarksgame-2014-08-31/{}{}'.format(x,ext)), ext) for x in deep_list for ext in ext_list]

In [69]:
# for key in snippit_dict:
#     with codecs.open(,'r',encoding='Windows 1252') as f:
#         data.append(f.read())

In [84]:
final_list = []
for x,y in this:
    if x != []:
        final_list.append((x,y))

In [85]:
snippit_dict = {}
for itm, idx in final_list:
    snippit_dict[idx] = itm


In [86]:
for key in snippit_dict:
    lan_list = []
    for string in snippit_dict[key]:
        with open(string) as f:
            lan_list.append(f.read())
    snippit_dict[key] = lan_list

In [93]:
snippit_dict['c'] = snippit_dict['.c'] + snippit_dict['.gcc']
snippit_dict['ruby'] = snippit_dict['.jruby'] + snippit_dict['.yarv']
snippit_dict['python'] = snippit_dict['.python3']
snippit_dict['php'] = snippit_dict['.hack'] + snippit_dict['.php']
snippit_dict['scala'] = snippit_dict['.scala']
snippit_dict['ocaml'] = snippit_dict['.ocaml']
snippit_dict['c#'] = snippit_dict['.csharp']
snippit_dict['lisp'] = snippit_dict['.sbcl']
snippit_dict['haskell'] = snippit_dict['.ghc']
snippit_dict['clojure'] = snippit_dict['.clojure']
snippit_dict['java'] = snippit_dict['.java']
snippit_dict['javascript'] = snippit_dict['.javascript']
snippit_dict['perl'] = snippit_dict['.perl']
snippit_dict['scheme'] = snippit_dict['.racket']

In [95]:
for string in ext_list:
    del snippit_dict[string]

In [110]:
X = []
y = []
for key in snippit_dict:
    y.extend([key for _ in range(len(snippit_dict[key]))])
    X.extend(snippit_dict[key])

In [115]:
symbol_list = '''(
)
def
end
:
#
@
let
elsif
elif
$
my
sub
use
{
}
*
ui
void
mul
done
for
while
;
(def
){
) {
public
static
native
<?
(define
(declaim
'''.split()

In [117]:
def percent_symbol(text,symbol):
    cleaned = re.sub(r'\s','',text)
    ret = len(re.findall(r''+re.escape(symbol),cleaned))
    return ret/len(cleaned)
    


In [120]:
class FunctionFeaturizer(TransformerMixin):
    def __init__(self, repeater, *featurizers, sym_list=None):
        self.repeater = repeater
        self.featurizers = featurizers
        self.sym_list=sym_list
        
    def fit(self, X, y=None):
        '''All SciKit-learn compatible transformers and classifiers have the same
        interface. `fit` should always return the same object (self)'''
        return self
    
    def transform(self, X):
        '''Given a list of original data, return a list of feature vectors'''
        feature_vectors = []
        if self.sym_list:
            for x in X:
                feature_vector = [self.repeater(x,sym) for sym in self.sym_list] 
                feature_vectors.append(feature_vector)
        
        return np.array(feature_vectors)


In [121]:
pipe = make_pipeline(FunctionFeaturizer(percent_symbol,sym_list=symbol_list),DecisionTreeClassifier())

In [133]:
X_train, X_test, y_train, y_test = train_test_split(X,y)

In [134]:
pipe.fit(X_train,y_train)

Pipeline(steps=[('functionfeaturizer', <__main__.FunctionFeaturizer object at 0x113df1630>), ('decisiontreeclassifier', DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            random_state=None, splitter='best'))])

In [135]:
print(classification_report(pipe.predict(X_test), y_test))


             precision    recall  f1-score   support

          c       1.00      1.00      1.00        14
         c#       1.00      0.83      0.91        12
    clojure       0.90      1.00      0.95         9
    haskell       0.57      1.00      0.73         4
       java       0.00      0.00      0.00         0
 javascript       1.00      1.00      1.00         6
       lisp       1.00      1.00      1.00        10
      ocaml       1.00      0.92      0.96        13
       perl       1.00      1.00      1.00         7
        php       0.97      0.97      0.97        37
     python       1.00      0.83      0.91        12
       ruby       0.92      1.00      0.96        11
      scala       1.00      0.71      0.83         7
     scheme       0.88      1.00      0.93         7

avg / total       0.96      0.95      0.95       149



/Users/Shea_Wolfe/tiy/week5/w5d4/.direnv/python-3.4.3/lib/python3.4/site-packages/sklearn/metrics/classification.py:960: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [137]:
pipe.score(X_test,y_test)

0.94630872483221473

In [139]:
pipe.predict([''',,(((((((((((<<<,,,,,<>>>>>)))))))))))'''])

array(['php'], 
      dtype='<U10')

In [ ]:
percent_symbol('()()((himynameisbob))','(')

In [ ]:
symbol_list[:5]

In [ ]:
this = FunctionFeaturizer(percent_symbol,sym_list=symbol_list)

In [ ]:
test = this.transform([',,daj;kj;3kj2rme/.ar;3lkj()(#*#$)<>','kfdjs;fj32hr343())()()()'])

In [ ]:
test

In [ ]:
len(symbol_list)

In [ ]:
class FunctionFeaturizer(TransformerMixin):
    def __init__(self, *featurizers):
        self.featurizers = featurizers
        
    def fit(self, X, y=None):
        '''All SciKit-learn compatible transformers and classifiers have the same
        interface. `fit` should always return the same object (self)'''
        return self
    
    def transform(self, X, sym_list=None):
        '''Given a list of original data, return a list of feature vectors'''
        feature_vectors = []
        if sym_list:
            for sym in sym_list:
                for x in X:
                    feature_vector = [f(x,sym) for f in self.featurizers[0]]
                    feature_vectors.append(feature_vector)
        else:
            for x in X:
                feature_vector = [f(x) for f in self.featurizers]
                feature_vectors.append(feature_vector)
        
        return np.array(feature_vectors)